In [ ]:
# Uniform-cost search
import heapq
import psutil
import time

class QueenBoard:
    def __init__(self, n):
        self.n = n
        self.board = [0] * n
        self.heuristic = self.calculate_heuristic()

    def calculate_heuristic(self):
        attacks = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if self.board[i] == self.board[j] or abs(self.board[i] - self.board[j]) == j - i:
                    attacks += 1
        return attacks

    def move_queen(self, col, new_row):
        old_row = self.board[col]
        if old_row != new_row:
            self.board[col] = new_row
            self.heuristic = self.calculate_heuristic()

    def get_next_states(self):
        next_states = []
        for col in range(self.n):
            for row in range(self.n):
                if self.board[col] != row:
                    new_state = QueenBoard(self.n)
                    new_state.board = self.board.copy()
                    new_state.move_queen(col, row)
                    next_states.append(new_state)
        return next_states

    def __lt__(self, other):
        return self.heuristic < other.heuristic

def uniform_cost_search(n):
    start_state = QueenBoard(n)
    if start_state.heuristic == 0:
        return start_state.board, 0.0

    frontier = [(0, start_state)]
    explored = set()
    while frontier:
        current_cost, current_state = heapq.heappop(frontier)
        if current_state.heuristic == 0:
            return current_state.board, current_cost

        explored.add(tuple(current_state.board))
        for next_state in current_state.get_next_states():
            if tuple(next_state.board) not in explored:
                heapq.heappush(frontier, (current_cost + 1, next_state))

    return None, 0.0

def memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    memory_usage = memory_info.rss / 1024 / 1024  # Convert to MB
    return memory_usage

def print_board(board):
    n = len(board)
    for row in board:
        board_row = ['Q' if i == row else '*' for i in range(n)]
        print(" ".join(board_row))

def main():
    n = int(input("Enter the value of n: "))
    start_time = time.time()
    solution, time_taken = uniform_cost_search(n)
    end_time = time.time()

    if solution is None:
        print("No solution found.")
    else:
        print("Solution:", solution)
        print("Board:")
        print_board(solution)

    print("Time taken:", end_time - start_time, "seconds")
    print("Memory used:", memory_usage(), "MB")

if __name__ == "__main__":
    main()

In [ ]:
# Graph-search A with MIN-CONFLICT heuristic
import time
import psutil

# Hàm tính số cặp quân hậu tấn công nhau trong một trạng thái
def count_attacking_pairs(board):
    n = len(board)
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            if board[i] == board[j] or abs(i - j) == abs(board[i] - board[j]):
                count += 1
    return count

# Hàm tìm giải pháp bằng thuật toán A*
def solve_n_queens(n):
    start_time = time.time()

    # Khởi tạo trạng thái ban đầu
    initial_state = [0] * n

    # Hàm heuristic: Số cặp quân hậu tấn công nhau
    def heuristic(board):
        return count_attacking_pairs(board)

    # Hàm A* search
    def A_star_search():
        frontier = [(heuristic(initial_state), initial_state)]
        explored = set()

        while frontier:
            # Lấy trạng thái có cost heuristic nhỏ nhất
            _, state = frontier.pop(0)

            # Kiểm tra xem trạng thái hiện tại có phải là giải pháp
            if heuristic(state) == 0:
                return state

            explored.add(tuple(state))

            # Tạo các trạng thái kế tiếp từ trạng thái hiện tại
            successors = generate_successors(state)

            for successor in successors:
                # Kiểm tra xem trạng thái kế tiếp đã được khám phá hay chưa
                if tuple(successor) not in explored:
                    frontier.append((heuristic(successor), successor))

            # Sắp xếp frontier theo cost heuristic
            frontier.sort(key=lambda x: x[0])

    # Tìm giải pháp
    solution = A_star_search()

    end_time = time.time()
    execution_time = end_time - start_time

    # Tính toán memory được sử dụng
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Memory usage in MB

    return solution, execution_time, memory_usage

# Hàm kiểm tra xem quân hậu tại (row, col) có bị tấn công bởi các quân hậu khác không
def is_attacked(board, row, col):
    # Kiểm tra trên cùng một cột
    for i in range(row):
        if board[i] == col:
            return True
    # Kiểm tra trên hai đường chéo
    for i in range(row):
        if abs(row - i) == abs(col - board[i]):
            return True
    return False

# Hàm tạo các trạng thái kế tiếp từ một trạng thái hiện tại
def generate_successors(board):
    n = len(board)
    successors = []
    for col in range(n):
        for row in range(n):
            if board[row] != col:
                successor = list(board)
                successor[row] = col
                successors.append(successor)
    return successors

# Hàm in ra bảng giải pháp
def print_solution(solution):
    n = len(solution)
    for row in range(n):
        line = ""
        for col in range(n):
            if solution[row] == col:
                line += "Q "
            else:
                line += "* "
        print(line)

def main():
    n = int(input("Enter the value of n: "))
    start_time = time.time()
    solution, execution_time, memory_usage = solve_n_queens(n)
    end_time = time.time()

    if solution is None:
        print("No solution found.")
    else:
        print("Solution:")
        print_solution(solution)

    print("Execution Time:", execution_time, "s")
    print("Memory Usage:", memory_usage, "MB")

if __name__ == "__main__":
    main()

In [ ]:
#Genetic algorithm with the objective function defined from the above heuristic
import random
import time
import psutil

# Hàm tính toán giá trị fitness cho một cá thể
def fitness(board):
    clashes = 0
    n = len(board)

    # Kiểm tra xem có xung đột nào không xảy ra theo hàng ngang và đường chéo
    for i in range(n):
        for j in range(i + 1, n):
            if board[i] == board[j] or board[i] - board[j] == i - j or board[i] - board[j] == j - i:
                clashes += 1

    # Fitness = 1 / (số lượng xung đột + 1)
    fitness_value = 1 / (clashes + 1)
    return fitness_value


# Hàm tạo ra một quần thể (population) ban đầu
def initialize_population(size, n):
    population = []
    for _ in range(size):
        board = [random.randint(0, n - 1) for _ in range(n)]
        population.append(board)
    return population


# Hàm di chuyển một quân hậu sang một ô khác trong cùng một cột
def move_queen(board):
    n = len(board)
    moved_board = board[:]
    # Chọn một quân hậu ngẫu nhiên để di chuyển
    queen_index = random.randint(0, n - 1)
    # Di chuyển quân hậu đến một ô khác trong cùng một cột
    new_row = random.randint(0, n - 1)
    moved_board[queen_index] = new_row
    return moved_board


# Hàm chọn lọc cá thể trong quần thể cho thế hệ tiếp theo
def selection(population):
    # Sử dụng phương pháp chọn lọc tỷ lệ để chọn lọc cá thể
    fitness_values = [fitness(board) for board in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness_value / total_fitness for fitness_value in fitness_values]
    selected_boards = random.choices(population, probabilities, k=len(population))
    return selected_boards


# Hàm thực thi thuật toán di truyền để giải bài toán N-Queen
def genetic_algorithm(n, population_size, generations):
    population = initialize_population(population_size, n)

    for _ in range(generations):
        # Chọn lọc quần thể hiện tại
        selected_population = selection(population)

        new_population = []

        while len(new_population) < population_size:
            # Di chuyển một quân hậu của một cá thể
            parent = random.choice(selected_population)
            child = move_queen(parent)

            new_population.append(child)

        population = new_population

    # Tìm cá thể tốt nhất trong quần thể cuối cùng
    best_board = max(population, key=fitness)
    best_fitness = fitness(best_board)

    return best_board, best_fitness


# Hàm in bảng board với ký hiệu Q và *
def print_board(board):
    n = len(board)
    for i in range(n):
        row = ['Q' if j == board[i] else '*' for j in range(n)]
        print(' '.join(row))
    print()


def main():
    n = int(input("Nhập số lượng quân hậu (n): "))
    population_size = 100  # Kích thước quần thể
    generations = 100  # Số lượng thế hệ

    start_time = time.time()
    solution, fitness_value = genetic_algorithm(n, population_size, generations)
    end_time = time.time()

    print("Solution:", solution)
    print("Board:")
    print_board(solution)

    elapsed_time = end_time - start_time
    memory_usage = psutil.Process().memory_info().rss / 1024 / 1024

    print("Time (s):", elapsed_time)
    print("Memory (MB):", memory_usage)


if __name__ == "__main__":
    main()
